# Sample Sentiment Analysis

In [1]:
!pip install nest_asyncio
from logging.handlers import TimedRotatingFileHandler
import logging.config
from psenti import SentimentAnalysis, SentimentConnection, Document
from sklearn import metrics
from datetime import datetime
import socket
import json

user_name = socket.gethostname()
host = '127.0.0.1'
port = 5000

import nest_asyncio
import asyncio
nest_asyncio.apply(loop=asyncio.get_event_loop())

logger = logging.getLogger('JupyterUI')
logFormatter = logging.Formatter('%(asctime)s - [%(thread)s] [%(threadName)s]- %(name)s - %(levelname)s - %(message)s')
logger.setLevel(logging.DEBUG)

console = logging.StreamHandler()
console.setFormatter(logFormatter)
console.setLevel(logging.DEBUG)

logger.addHandler(console)

In [3]:
%%time

connection = SentimentConnection(host=host, port=port, client_id=user_name)
logger.info(f'Supported domains')
for domain in connection.supported_domains:    
    logger.info(f'Domain: [{domain}]')

2020-06-17 22:51:35,183 - [87544] [MainThread]- JupyterUI - INFO - Supported domains


AttributeError: 'SentimentConnection' object has no attribute 'supported_domains'

# Standard Lexicon Sentiment Analysis

### Setup results Processing

In [3]:
# simplified reviews
reviews = ['I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
           'I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.']

# reviews with additional metadata - date, author, id
documents = [
        Document('I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
                 '1',
                 'Ben'),
        Document('I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.',
                 '2',
                 'Ben',
                 datetime(1995, 5, 2))
        ]

def process_result(domain, result):
    if result['Stars'] is None:
        logger.warning(f'{domain}: [{result["Id"]}] No Sentinent')
    else:
        logger.info(f'{domain}: [{result["Id"]}] Sentinment Stars: {result["Stars"]:1.2f}')     
        
    for sentence in result['Sentences']:
        for word in sentence['Words']:
            if 'Value' in word:
                logger.info(f"[{word['Span']}] Sentiment: {word['Value']:1.2f}")     


### Analyse using standard sentiment lexicon

In [4]:
analysis = SentimentAnalysis(connection)
analysis.on_message.subscribe(lambda result: process_result(domain, result))
analysis.detect_sentiment_text(reviews)

2020-05-29 10:29:30,209 - [49832] [MainThread]- JupyterUI - INFO - market: [039fc641-ef3a-49d8-bf70-024a7cc96040] Sentinment Stars: 3.58
2020-05-29 10:29:30,209 - [49832] [MainThread]- JupyterUI - INFO - [love] Sentiment: 3.00
2020-05-29 10:29:30,210 - [49832] [MainThread]- JupyterUI - INFO - [like] Sentiment: 2.00
2020-05-29 10:29:30,210 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -2.00
2020-05-29 10:29:30,211 - [49832] [MainThread]- JupyterUI - INFO - [fault] Sentiment: -2.00
2020-05-29 10:29:30,211 - [49832] [MainThread]- JupyterUI - INFO - [fast] Sentiment: 1.00
2020-05-29 10:29:30,212 - [49832] [MainThread]- JupyterUI - INFO - market: [a3310b0c-ea64-48a3-8dfd-c1da91c337b2] Sentinment Stars: 3.68
2020-05-29 10:29:30,213 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -2.00
2020-05-29 10:29:30,213 - [49832] [MainThread]- JupyterUI - INFO - [cute] Sentiment: 2.00
2020-05-29 10:29:30,214 - [49832] [MainThread]- JupyterUI - INFO - [cheap] Sentiment: -3.00
20

### Analyse using custom domain lexicons

In [5]:
for domain in connection.supported_domains:
    # analyse using custom domain lexicons
    analysis = SentimentAnalysis(connection, domain=domain)
    analysis.on_message.subscribe(lambda result: process_result(domain, result))
    analysis.detect_sentiment(documents)

2020-05-29 10:29:31,081 - [49832] [MainThread]- JupyterUI - INFO - TwitterMarket: [2] Sentinment Stars: 4.54
2020-05-29 10:29:31,082 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -2.00
2020-05-29 10:29:31,082 - [49832] [MainThread]- JupyterUI - INFO - [cheap] Sentiment: 1.81
2020-05-29 10:29:31,083 - [49832] [MainThread]- JupyterUI - INFO - [easy] Sentiment: 2.00
2020-05-29 10:29:31,083 - [49832] [MainThread]- JupyterUI - INFO - [up] Sentiment: 2.00
2020-05-29 10:29:31,084 - [49832] [MainThread]- JupyterUI - INFO - TwitterMarket: [1] Sentinment Stars: 1.00
2020-05-29 10:29:31,084 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -2.00
2020-05-29 10:29:31,085 - [49832] [MainThread]- JupyterUI - INFO - [fault] Sentiment: -2.00
2020-05-29 10:29:31,193 - [49832] [MainThread]- JupyterUI - INFO - TwitterTrump: [1] Sentinment Stars: 1.44
2020-05-29 10:29:31,194 - [49832] [MainThread]- JupyterUI - INFO - [like] Sentiment: -1.26
2020-05-29 10:29:31,195 - [49832] [MainThr

### Analysis Using Custom lexicon

In [6]:
lexicon = {'love': -3, 'bad oder': 3, 'bad': -1} # only this lexicon will be in use
analysis = SentimentAnalysis(connection, lexicon=lexicon)
analysis.on_message.subscribe(lambda result: process_result(domain, result))
analysis.detect_sentiment_text(reviews)

2020-05-29 10:29:31,441 - [49832] [MainThread]- JupyterUI - INFO - market: [fd32ee5a-9a31-4de1-85c8-8c1e71c93b83] Sentinment Stars: 5.00
2020-05-29 10:29:31,442 - [49832] [MainThread]- JupyterUI - INFO - [bad oder] Sentiment: 3.00
2020-05-29 10:29:31,443 - [49832] [MainThread]- JupyterUI - INFO - market: [79d07d67-99de-48b5-98c9-dab8777b8a8a] Sentinment Stars: 1.00
2020-05-29 10:29:31,443 - [49832] [MainThread]- JupyterUI - INFO - [love] Sentiment: -3.00
2020-05-29 10:29:31,444 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -1.00


### Modification of existing lexicon

In [7]:
lexicon = {'love': -3, 'bad oder': 3} # lexicon modifaction

# without modification
analysis = SentimentAnalysis(connection, domain='medical')
analysis.on_message.subscribe(lambda result: process_result('Medical', result))
analysis.detect_sentiment(documents)

# with modification
analysis = SentimentAnalysis(connection, domain='medical', lexicon=lexicon, adjust_lexicon=True)
analysis.on_message.subscribe(lambda result: process_result('Adjusted Medical', result))
analysis.detect_sentiment(documents)

2020-05-29 10:29:31,544 - [49832] [MainThread]- JupyterUI - INFO - Medical: [1] Sentinment Stars: 3.95
2020-05-29 10:29:31,545 - [49832] [MainThread]- JupyterUI - INFO - [love] Sentiment: 1.85
2020-05-29 10:29:31,546 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -1.99
2020-05-29 10:29:31,546 - [49832] [MainThread]- JupyterUI - INFO - [fast] Sentiment: 1.99
2020-05-29 10:29:31,547 - [49832] [MainThread]- JupyterUI - INFO - Medical: [2] Sentinment Stars: 4.33
2020-05-29 10:29:31,547 - [49832] [MainThread]- JupyterUI - INFO - [bad] Sentiment: -1.99
2020-05-29 10:29:31,548 - [49832] [MainThread]- JupyterUI - INFO - [cheap] Sentiment: 3.00
2020-05-29 10:29:31,548 - [49832] [MainThread]- JupyterUI - INFO - [easy] Sentiment: 2.00
2020-05-29 10:29:31,611 - [49832] [MainThread]- JupyterUI - INFO - Adjusted Medical: [2] Sentinment Stars: 5.00
2020-05-29 10:29:31,611 - [49832] [MainThread]- JupyterUI - INFO - [bad oder] Sentiment: 3.00
2020-05-29 10:29:31,612 - [49832] [MainThread]-